In [1]:
from affiliation.generics import convert_vector_to_events
from affiliation.metrics import pr_from_events
import numpy as np
import pandas as pd
import os
from sklearn import metrics
from sklearn.metrics import roc_auc_score, average_precision_score, confusion_matrix, roc_curve, precision_recall_curve

In [4]:
path = os.path.join('results/ours', 'smd') #'msl-smap'
# sources = ['F-5', 'P-10', 'D-14', 'C-1', 'A-7', 'P-2', 'E-8', 'D-7']
sources = ['1-1', '2-3', '3-7', '1-5']
file_list = os.listdir(path)                        
file_list = sorted(file_list)
best_f1_sum, aff_f1_sum, best_pre_sum, best_rec_sum, aff_pre_sum, aff_rec_sum = [],[],[],[],[],[]
evals = []

for source in sources:
    results = []
    for filename in file_list:
        if source+'-' in filename:
#             print(filename)
            df_test = pd.read_csv(path+'/'+filename+'/predictions_test_target.csv')

            roc_auc = roc_auc_score(df_test['y'], df_test['y_pred'])
    #         print('AU-ROC : ', roc_auc)

            pr_auc = average_precision_score(df_test['y'], df_test['y_pred'])
    #         print('AU-PR : ', pr_auc)

            fpr, tpr, thresholds = roc_curve(df_test['y'], df_test['y_pred'], pos_label=1)
            precision, recall, thresholds = precision_recall_curve(df_test['y'], df_test['y_pred'], pos_label=1)


            res = pd.DataFrame()
            res['pre'] = precision
            res['rec'] = recall
            res['f1'] = 2*res['pre']*res['rec'] / (res['pre']+res['rec'])
            best_idx = res['f1'].argmax()
            best_f1 = res['f1'][best_idx]
            best_pre = res['pre'][best_idx]
            best_rec = res['rec'][best_idx]
            best_thr = thresholds[best_idx]
            best_f1_sum.append(best_f1)
            best_pre_sum.append(best_pre)
            best_rec_sum.append(best_rec)
#             print('Bst Pre :', round(best_pre, 4), 'Bst Rec:', round(best_rec, 4), 'Bst f1 :',round(best_f1,4))


#             threshold = 0.2
            vector_score = df_test['y_pred'].values
            vector_pred = (vector_score >= best_thr).astype(int)
            vector_gt = df_test['y'].values
            events_pred = convert_vector_to_events(vector_pred)
            events_gt = convert_vector_to_events(vector_gt)
            Trange = (0, len(vector_pred))

            final_res = pr_from_events(events_pred, events_gt, Trange)
            aff_pre = final_res['precision']
            aff_rec = final_res['recall']
            aff_f1 = 2*aff_pre*aff_rec / (aff_pre+aff_rec)
            aff_f1_sum.append(aff_f1)
            aff_pre_sum.append(aff_pre)
            aff_rec_sum.append(aff_rec)
#             print('Aff Pre :', round(aff_pre, 4), 'Aff Rec:', round(aff_rec, 4), 'Aff f1 :',round(aff_f1,4))

            tmp_res = {
            'src-trg': filename,
            'Bst Pre': round(best_pre, 3),
            'Bst Rec': round(best_rec, 3),
            'Bst f1': round(best_f1,3),
            'Aff Pre': round(aff_pre, 3),
            'Aff Rec': round(aff_rec, 3),
            'Aff f1': round(aff_f1,3)
            }

            results.append(tmp_res)
            if not(os.path.exists('affliation_res')): os.makedirs('affliation_res')
            pd.DataFrame(results).to_csv('affliation_res/aff-'+source+'.csv', index=False)

#     print("AVG f1 :", round(best_f1_sum / total_tests, 4), 
#           "AVG pre :", round(best_pre_sum / total_tests, 4), 
#           "AVG rec :", round(best_rec_sum / total_tests, 4))
#     print("AVG AF f1 :", round(aff_f1_sum / total_tests, 4), 
#           "AVG AF pre :", round(aff_pre_sum / total_tests, 4), 
#           "AVG AF rec :", round(aff_rec_sum / total_tests, 4))
    tmp_evals = {
            'src': source,
            'AVG pre': round(np.mean(best_pre_sum), 3),
            'STD pre': round(np.std(best_pre_sum), 3),
            'AVG rec': round(np.mean(best_rec_sum), 3),
            'STD rec': round(np.std(best_rec_sum), 3),
            'AVG f1': round(np.mean(best_f1_sum), 3),
            'STD f1': round(np.std(best_f1_sum), 3),
            'AVG AF pre': round(np.mean(aff_pre_sum), 3),
            'STD AF pre': round(np.std(aff_pre_sum), 3),
            'AVG AF rec': round(np.mean(aff_rec_sum), 3),
            'STD AF rec': round(np.std(aff_rec_sum), 3),
            'AVG AF f1': round(np.mean(aff_f1_sum), 3),
            'STD AF f1': round(np.std(aff_f1_sum), 3)
            }

    evals.append(tmp_evals)

pd.DataFrame(evals).to_csv('aff-smd.csv', index=False)